In [1]:
import juliacall
import numpy as np
import fit_modeler
import evolve_binary
import pandas as pd

np.set_printoptions(legacy='1.13')

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
q1
q
b
c
y
x


In [2]:
population = pd.read_csv('/Users/sray/Documents/Saavik_Barry/test_mcfacts/evolve_binary/output_bh_binary_55.dat', delimiter=' ', header = 0)

In [3]:
population

,orb_a_1,orb_a_2,mass_1,mass_2,spin_1,spin_2,spin_angle_1,spin_angle_2,bin_sep,bin_orb_a,...,time_merged,bin_ecc,gen_1,gen_2,bin_orb_ang_mom,bin_orb_inc,bin_orb_ecc,gw_freq,gw_strain,id_num
0,44565.608813,44698.568660,10.510475,22.847318,-0.125363,-0.027870,2.037367,1.576808,131.068672,45041.809097,...,0.0,0.860597,1.0,1.0,1.0,0.741368,0.011,2.429514e-10,2.532940e-31,208.0
1,27999.768798,28039.195607,19.410351,26.807664,0.037107,0.063074,0.762779,0.404380,35.933705,28022.637384,...,0.0,0.041883,1.0,1.0,1.0,0.180225,0.011,1.992149e-09,5.732680e-30,210.0
2,13354.941938,13411.676127,27.850180,11.038404,0.697989,-0.009064,0.000000,1.094391,51.337943,13325.869836,...,0.0,0.603960,2.0,1.0,1.0,0.000000,0.010,1.070095e-09,1.737455e-30,211.0
3,21310.846852,21397.750026,53.381132,13.547682,0.686014,0.038693,0.000000,0.412239,86.903174,21328.437730,...,0.0,0.871070,2.0,1.0,1.0,0.000000,0.010,6.374148e-10,1.863523e-30,212.0


In [4]:

# No longer needed since new surrogate model intakes individual mass values and not the mass ratio

'''#sorting mass ratios
mass_1 = []
mass_2 = []
for i in range(len(population['mass_1'])):
    if population['mass_1'][i] > population['mass_2'][i]:
        mass_1.append(population['mass_1'][i])
        mass_2.append(population['mass_2'][i])
    else:
        mass_1.append(population['mass_2'][i])
        mass_2.append(population['mass_1'][i])
print(mass_1)
print(mass_2)'''

"#sorting mass ratios\nmass_1 = []\nmass_2 = []\nfor i in range(len(population['mass_1'])):\n    if population['mass_1'][i] > population['mass_2'][i]:\n        mass_1.append(population['mass_1'][i])\n        mass_2.append(population['mass_2'][i])\n    else:\n        mass_1.append(population['mass_2'][i])\n        mass_2.append(population['mass_1'][i])\nprint(mass_1)\nprint(mass_2)"

In [5]:
test = [[], [], []]
for i in range(len(test)):
    #test[i].append(i)
    test[0].append(i**1)
    #y
    test[1].append(i**2)
    #z
    test[2].append(i**3)
print(test)

[[0, 1, 2], [0, 1, 4], [0, 1, 8]]


In [6]:
# Converting spin_1 magnitude values into cartesian values
chiA_components = [[] for _ in range(len(population['spin_1']))] 
theta = 0.0

for i in range(len(chiA_components)):
    #x component
    chiA_components[i].append(population['spin_1'][i] * np.cos(theta))
    #y
    chiA_components[i].append(population['spin_1'][i] * np.sin(theta))
    #z
    chiA_components[i].append(np.sqrt((chiA_components[i][0])**2.0 + (chiA_components[i][1])**2.0))
    
print(chiA_components)

[[-0.12536294050404401, -0.0, 0.12536294050404401], [0.037107405922282927, 0.0, 0.037107405922282927], [0.69798866435038365, 0.0, 0.69798866435038365], [0.68601396020331862, 0.0, 0.68601396020331862]]


In [7]:
# Converting spin_2 magnitude values into cartesian values
chiA_components = [[] for _ in range(len(population['spin_2']))] 
theta = 0.0

for i in range(len(chiA_components)):
    #x component
    chiA_components[i].append(population['spin_2'][i] * np.cos(theta))
    #y
    chiA_components[i].append(population['spin_2'][i] * np.sin(theta))
    #z
    chiA_components[i].append(np.sqrt((chiA_components[i][0])**2.0 + (chiA_components[i][1])**2.0))
    
print(chiA_components)

[[-0.027869839323211215, -0.0, 0.027869839323211215], [0.063073744124955256, 0.0, 0.063073744124955256], [-0.0090636927188756447, -0.0, 0.0090636927188756447], [0.038693434531554861, 0.0, 0.038693434531554861]]


In [8]:
population['mass_1'][0]

10.510474631893103

In [9]:
print(population['spin_angle_1'][0] - population['spin_angle_2'][0])

0.460558847882


In [10]:
for i in range(len(population['mass_1'])):
    mass_1 = population['mass_1'][i]
    mass_2 = population['mass_2'][i]
    spin_1 = population['spin_1'][i]
    spin_2 = population['spin_2'][i]
    spin_angle_1 = population['spin_angle_1'][i]
    spin_angle_2 = population['spin_angle_2'][i]
    phi_12 = population['spin_angle_2'][i] - population['spin_angle_1'][i]
    bin_sep = population['bin_sep'][i] * (1.e8 / (mass_1 + mass_2))
    bin_orb_inc = [0, 0, 1]

In [11]:
print("sur")
surrogate = fit_modeler.GPRFitters.read_from_file(f"surrogate.joblib")

sur


In [12]:
parameters = [mass_1, mass_2, spin_1, spin_2, spin_angle_1, spin_angle_2, phi_12, bin_sep, bin_orb_inc]
for i in parameters:
    print(i)
    print("")

53.3811315668

13.5476823295

0.686013960203

0.0386934345316

0.0

0.412238819355

0.412238819355

129844186.074

[0, 0, 1]



In [13]:
print("e")
M_f, spin_f, v_f = evolve_binary.evolve_binary(
    mass_1,
    mass_2,
    spin_1,
    spin_2,
    spin_angle_1,
    spin_angle_2,
    phi_12,
    bin_sep,
    bin_orb_inc,
    surrogate,
    True,
)

print("M_f = ", M_f)
print("spin_f = ", spin_f)
print("v_f = ", v_f)

e
start
Evolving binary...
chi_1
chi_2
mass check


┌ Warning: Interrupted. Larger maxiters is needed. If you are using an integrator for non-stiff ODEs or an automatic switching algorithm (the default), you may want to consider using a method for stiff equations. See the solver pages for more details (e.g. https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/#Stiff-Problems).
└ @ SciMLBase ~/.julia/packages/SciMLBase/NtgCQ/src/integrator_interface.jl:589


final
real final
Evolving binary through merger with surrogate...
real ^2 final
M_f =  65.4291225742
spin_f =  [  4.38346003e-01   3.82077660e-01   2.03482924e+03]
v_f =  [ -1.07519672e-04   5.42113865e-04   6.99717987e-07]


In [14]:
mass_1 = population['mass_1']
mass_2 = population['mass_2']
spin_1 = population['spin_1']
spin_2 = population['spin_2']
spin_angle_1 = population['spin_angle_1']
spin_angle_2 = population['spin_angle_2']
phi_12 = population['spin_angle_2'] - population['spin_angle_1']
bin_sep = population['bin_sep'] * (1.e8 / (mass_1 + mass_2))
bin_orb_inc = [0, 0, 1]

mass_final = []
spin_final = []
velocity_final =[]

In [15]:
import time

In [16]:
for i in range(len(mass_1)):
    start = time.time()
    M_f, spin_f, v_f = evolve_binary.evolve_binary(
        mass_1[i],
        mass_2[i],
        spin_1[i],
        spin_2[i],
        spin_angle_1[i],
        spin_angle_2[i],
        phi_12[i],
        bin_sep[i],
        bin_orb_inc,
        surrogate,
        True,
    )
    end = time.time()
    
    run_time = end - start

    mass_final.append(M_f)
    spin_final.append(spin_f)
    velocity_final.append(v_f)
    
    print("M_f = ", M_f)
    print("spin_f = ", spin_f)
    print("v_f = ", v_f)
    print("Merger took ", run_time, " seconds")

start
Evolving binary...
chi_1
chi_2
mass_1 < mass_2! Flipping black hole labels.
mass check
final
real final
Evolving binary through merger with surrogate...
real ^2 final
M_f =  32.1311587785
spin_f =  [ -2.78151394e-02   7.87750535e-03   6.27982820e+02]
v_f =  [ -5.45305527e-04   4.69460879e-05  -3.17074791e-07]
Merger took  70.17285394668579  seconds
start
Evolving binary...
chi_1
chi_2
mass_1 < mass_2! Flipping black hole labels.
mass check


┌ Warning: Interrupted. Larger maxiters is needed. If you are using an integrator for non-stiff ODEs or an automatic switching algorithm (the default), you may want to consider using a method for stiff equations. See the solver pages for more details (e.g. https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/#Stiff-Problems).
└ @ SciMLBase ~/.julia/packages/SciMLBase/NtgCQ/src/integrator_interface.jl:589
┌ Warning: Interrupted. Larger maxiters is needed. If you are using an integrator for non-stiff ODEs or an automatic switching algorithm (the default), you may want to consider using a method for stiff equations. See the solver pages for more details (e.g. https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/#Stiff-Problems).
└ @ SciMLBase ~/.julia/packages/SciMLBase/NtgCQ/src/integrator_interface.jl:589


final
real final
Evolving binary through merger with surrogate...
real ^2 final
M_f =  44.0902141221
spin_f =  [ -3.17771191e-02   4.27412481e-02   1.30673072e+03]
v_f =  [ -1.74211236e-04   2.28311594e-04   7.00961742e-07]
Merger took  68.30069518089294  seconds
start
Evolving binary...
chi_1
chi_2
mass check
final
real final
Evolving binary through merger with surrogate...
real ^2 final
M_f =  37.6023128663
spin_f =  [  1.07589957e-01   4.91494961e-01   8.18189823e+02]
v_f =  [ -2.57728487e-04   5.26088575e-04   4.38005357e-06]
Merger took  88.90259003639221  seconds
start
Evolving binary...
chi_1
chi_2
mass check


┌ Warning: Interrupted. Larger maxiters is needed. If you are using an integrator for non-stiff ODEs or an automatic switching algorithm (the default), you may want to consider using a method for stiff equations. See the solver pages for more details (e.g. https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/#Stiff-Problems).
└ @ SciMLBase ~/.julia/packages/SciMLBase/NtgCQ/src/integrator_interface.jl:589
┌ Warning: Interrupted. Larger maxiters is needed. If you are using an integrator for non-stiff ODEs or an automatic switching algorithm (the default), you may want to consider using a method for stiff equations. See the solver pages for more details (e.g. https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/#Stiff-Problems).
└ @ SciMLBase ~/.julia/packages/SciMLBase/NtgCQ/src/integrator_interface.jl:589


final
real final
Evolving binary through merger with surrogate...
real ^2 final
M_f =  65.4291225742
spin_f =  [  4.38346003e-01   3.82077660e-01   2.03482924e+03]
v_f =  [ -1.07519672e-04   5.42113865e-04   6.99717987e-07]
Merger took  159.93965411186218  seconds


In [17]:
print(bin_sep)

0    3.929177e+08
1    7.774826e+07
2    1.320129e+08
3    1.298442e+08
dtype: float64


In [22]:
print(mass_final[-1])
print(spin_final[-1])
print(velocity_final[-1])

65.4291225742
[  4.38346003e-01   3.82077660e-01   2.03482924e+03]
[ -1.07519672e-04   5.42113865e-04   6.99717987e-07]
